In [10]:
!pip install crewai crewai[tools] google-generativeai python-dotenv
!pip install langchain_google_genai

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


#Import Libraries & Access Key:

In [11]:
import os
from google.colab import userdata
from crewai_tools import ScrapeWebsiteTool
from crewai import Agent, Task, Crew, Process, LLM

# --- Setup: Get API Keys ---
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    BROWSERLESS_API_KEY = userdata.get('BROWSERLESS_API_KEY')
except userdata.SecretNotFoundError as e:
    raise ValueError(f"Error loading secret: {e}. Please ensure both keys are set in Colab secrets.")

if not GOOGLE_API_KEY or not BROWSERLESS_API_KEY:
    raise ValueError("One or both API keys are missing.")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["BROWSERLESS_API_KEY"] = BROWSERLESS_API_KEY

# --- Agent and Crew Initialization ---
# FIX: Use the correct model name format for litellm.
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash", # Correct format for gemini provider
    api_key=os.environ["GOOGLE_API_KEY"]
)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


##Agent Building Process##

1- Define a Agents:

In [12]:
# Initialize website scraping tool
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role='Senior Job Researcher',
    goal="Scour a company's career page to find all available software engineering job listings",
    backstory="""You are an expert market researcher. Your specialty is finding and identifying relevant job opportunities from complex, dynamically-loaded career websites.
    You are a master at sifting through HTML to find the core job data.""",
    verbose=True,
    allow_delegation=False,
    tools=[scrape_tool],
    llm=gemini_llm  # Pass LLM to the agent directly
)

analyst = Agent(
    role='Job Data Analyst',
    goal='Extract key details from the provided job listings HTML content',
    backstory="""You are a meticulous data analyst. You can read raw HTML and extract specific information with perfect accuracy. You focus on job titles, locations, and application links, formatting them neatly.""",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm # Pass LLM to the agent directly
)

2- Design the Task:

In [13]:
# --- Define Tasks ---
target_url = "https://www.google.com/about/careers/applications/jobs/results/"
company_name = "Google"

research_task = Task(
    description=f'Visit the career page for {company_name} at the URL {target_url} and scrape all job content.',
    expected_output='The full, raw HTML content of the job listings page.',
    agent=researcher
)

analysis_task = Task(
    description="""From the provided HTML content, identify and extract the following for each
    software engineering job: Job Title, Location, and a direct URL link to the application page.
    Format the final output as a clean list of jobs.""",
    expected_output='A formatted list of software engineering jobs with their title, location, and application link.',
    agent=analyst
)

3- Assemble and Run the Crew:

In [14]:
# --- Create and Run the Crew ---
crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, analysis_task],
    verbose=True,
    process=Process.sequential
)

print("Starting the Job Aggregation Crew...")
result = crew.kickoff()
print("\n\n##################################")
print("## Final Report ##")
print(result)

Starting the Job Aggregation Crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ab1a60bf-2b89-447d-afb8-ba504df04f4b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Job Researcher                                                                                   │
│                                                                                                                 │
│  Task: Visit the career page for Google at the URL                                                              │
│  https://www.google.com/about/careers/applications/jobs/results/ and scrape all job content.                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Job Data Analyst                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is the formatted list of software engineering jobs with their title, location, and application link as    │
│  extracted from the provided HTML content:                                                                      │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, AI/ML                                                                │
│      **Location**: Zürich, Switzerland                                                                          │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Strategic Cloud Engineer, Application Modernization, Technical Delivery                     │
│      **Location**: Reston, VA, USA                                                                              │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Budgeting, Google Ads                                                │
│      **Location**: Mountain View, CA, USA                                                                       │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Google Cloud                                                         │
│      **Location**: Bengaluru, Karnataka, India ; Hyderabad, Telangana, India                                    │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Developer III, Infrastructure, Google Cloud Platforms                              │
│      **Location**: Waterloo, ON, Canada                                                                         │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Front End, Google Cloud                                              │
│      **Location**: Bengaluru, Karnataka, India                                                                  │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Borg, Cloud                                                          │
│      **Location**: Warsaw, Poland                                                                               │
│      **Application Link**: Not available in the provide

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 42f7efe5-dbb5-4981-a99e-c61b83a7f5d2                                                                     │
│  Agent: Job Data Analyst                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ab1a60bf-2b89-447d-afb8-ba504df04f4b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here is the formatted list of software engineering jobs with their title, location, and          │
│  application link as extracted from the provided HTML content:                                                  │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, AI/ML                                                                │
│      **Location**: Zürich, Switzerland                                                                          │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Strategic Cloud Engineer, Application Modernization, Technical Delivery                     │
│      **Location**: Reston, VA, USA                                                                              │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Budgeting, Google Ads                                                │
│      **Location**: Mountain View, CA, USA                                                                       │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Google Cloud                                                         │
│      **Location**: Bengaluru, Karnataka, India ; Hyderabad, Telangana, India                                    │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Developer III, Infrastructure, Google Cloud Platforms                              │
│      **Location**: Waterloo, ON, Canada                                                                         │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Front End, Google Cloud                                              │
│      **Location**: Bengaluru, Karnataka, India                                                                  │
│      **Application Link**: Not available in the provided HTML content.                                          │
│                                                                                                                 │
│  *   **Job Title**: Software Engineer III, Borg, Cloud                                                          │
│      **Location**: Warsaw, Poland                     



##################################
## Final Report ##
Here is the formatted list of software engineering jobs with their title, location, and application link as extracted from the provided HTML content:

*   **Job Title**: Software Engineer III, AI/ML
    **Location**: Zürich, Switzerland
    **Application Link**: Not available in the provided HTML content.

*   **Job Title**: Strategic Cloud Engineer, Application Modernization, Technical Delivery
    **Location**: Reston, VA, USA
    **Application Link**: Not available in the provided HTML content.

*   **Job Title**: Software Engineer III, Budgeting, Google Ads
    **Location**: Mountain View, CA, USA
    **Application Link**: Not available in the provided HTML content.

*   **Job Title**: Software Engineer III, Google Cloud
    **Location**: Bengaluru, Karnataka, India ; Hyderabad, Telangana, India
    **Application Link**: Not available in the provided HTML content.

*   **Job Title**: Software Developer III, Infrastructure, Goo